In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.3.0 cuda: True


In [2]:
## Load dataset
data_dir = '/home/hao/Research/apg_data'
data_path = data_dir + "/gmm/gmm_10size"
Data = []
for i in range(5):
    data = torch.from_numpy(np.load(data_path + '/obs_%d.npy' % ((i+1)*20))).float()
    Data.append(data)
K = 3 ## number of clusters
D = 2
NUM_HIDDEN_LOCAL = 32
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

PATH = 'apg-3run-60pts-10samples'

In [3]:
from model_operations import Init_models_test
# from model_operations_vae import Init_models
models = Init_models_test(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE, lr=1e-4, RESTORE=True, PATH=PATH)

In [4]:
from testing import Eval
from viz_mc import Viz_MC
from apg import APG_test

BATCH_SIZE = 5
Viz_INTERVAL = 4
FS = 15
TITLE_FS = 20
OB_MS = 5.0
MU_MARKER = 'X'
MU_MS = 100
OPACITY = 0.3
BOUND = 12
COLORS = ['#AA3377', '#EE7733', '#0077BB', '#009988']
DATA_PTR = 73

In [11]:
S = 1000
B = 20
MCMC_STEPS = 15
Test_Params = (S, B, DEVICE)
g = 4
def Test_ALL(objective, data_g, mcmc_steps, Test_Params):
    Metrics = { 'kl_in_eta' : [], 'kl_in_z' : [], 'kl_ex_eta' : [], 'kl_ex_z' : []}
    (S, B, DEVICE) = Test_Params
    EPS = torch.FloatTensor([1e-15]).log() ## EPS for KL between categorial distributions
    EPS = EPS.cuda().to(DEVICE) ## EPS for KL between categorial distributions
    NUM_DATASETS = data_g.shape[0]
    NUM_BATCHES = int((NUM_DATASETS / B))
    for step in range(NUM_BATCHES):
        ob = data_g[step*B : (step+1)*B]
        ob = shuffler(ob).repeat(S, 1, 1, 1)
        ob = ob.cuda().to(DEVICE)
        metrics = objective(models, ob, mcmc_steps, EPS)
        Metrics['kl_in_eta'].append(metrics['kl_in_eta'])
        Metrics['kl_ex_eta'].append(metrics['kl_ex_eta'])
        Metrics['kl_in_z'].append(metrics['kl_in_z'])
        Metrics['kl_ex_z'].append(metrics['kl_ex_z'])
    return Metrics

%time Metrics = Test_ALL(APG_test, Data[g], MCMC_STEPS, Test_Params)

CPU times: user 12min 23s, sys: 3min 41s, total: 16min 5s
Wall time: 16min 4s


In [10]:
data_order = '%d' %  ((g+1)*20)
for key in Metrics.keys():
    np.save('granular_%s-%s' % (data_order, key), torch.cat(Metrics[key], 0))

In [ ]:
visualizer = Viz_MC(K, Viz_INTERVAL, FS, TITLE_FS, OB_MS, OPACITY, BOUND, COLORS)
evaluator = Eval(models, K, D, BATCH_SIZE, CUDA, DEVICE)

# Metrics_sample = evaluator.Test_uniform(APG_test, Data, DATA_PTR, mcmc_steps=12, sample_size=1)
# visualizer.Plot_chains(Metrics_sample['data'], Metrics_sample['samples'], filename='gmm-samples-%d' % DATA_PTR)

In [12]:
for i in range(4):
    ex_z = np.load('granular_%d-kl_ex_z.npy' % (i+1)*20)

OSError: [Errno 36] File name too long: 'granular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npygranular_1-kl_ex_z.npy'

In [ ]:
# L = 1000
# Metrics = evaluator.Test_uniform(APG_test, Data, DATA_PTR, mcmc_steps=30, sample_size=L)
# visualizer.Plot_metrics(Metrics, sample_size=L, filename='gmm-metrics-LL-ESS-%d' % DATA_PTR)

In [ ]:
# Budget * Batch
DATA_PTR = 3
NUM_RUNS = 10
LogJs = []
ESSs = []
budgets = [[1000, 1], [500, 2], [200, 5], [100, 10], [50, 20], [20, 50], [10, 100], [5, 200]]
for r in range(NUM_RUNS):
    LogJ= []
    ESS = []
    for i, b in enumerate(budgets):
        samples = b[0]
        steps = b[1] - 1
        Metrics_converge = evaluator.Test_budget(APG_test, Data, DATA_PTR, mcmc_steps=steps, sample_size=samples)
        LogJ.append(torch.cat(Metrics_converge['log_joint'], 0).unsqueeze(0))
        ESS.append(torch.cat(Metrics_converge['ess'], 0).unsqueeze(0) / samples)
    LogJs.append(torch.cat(LogJ, 0).unsqueeze(0))
    ESSs.append(torch.cat(ESS, 0).unsqueeze(0))
    %time print('run : %d' % r)

LogJs = torch.cat(LogJs, 0)
ESSs = torch.cat(ESSs, 0)

In [ ]:
np.save('budget-log-joint', LogJs.data.numpy())
np.save('budget-ess', ESSs.data.numpy())

In [ ]:
LogJs.shape

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

COLORS = ['#AA3377', '#EE7733', '#0077BB', '#009988', '#CCBB44','#BBBBBB']



def Plot_metrics_v2(fs, LLs, ESSs, SSs, filename):
    num_cols = len(LLs[0])
    gs = gridspec.GridSpec(2, num_cols)
    gs.update(left=0.05 , bottom=0.05, right=0.95, top=0.95, wspace=0.2, hspace=0.05)
    fig = plt.figure(figsize=(fs, fs * 2 / num_cols))
    plt.rc('axes',edgecolor='#eeeeee')
    for col_ind in range(num_cols):
#         ax1 = fig.add_subplot(gs[0, col_ind])
#         ax2 = fig.add_subplot(gs[1, col_ind])
        print('N=%d' % ((col_ind+1)*20))
        for b, LL, in enumerate(LLs):
            ESS = ESSs[b]
            SS = SSs[b]
            print(ESS[col_ind][-1] / SS)
#             ax1.plot(LL[col_ind].data.numpy(), c=COLORS[b], marker='o')
#             ax2.plot(ESS[col_ind].data.numpy() / SS, '--', c=COLORS[b], alpha=0.4)
#     plt.savefig(filename + '.svg', dpi=300)
#     plt.savefig(filename + '.pdf')

In [ ]:
fs = 20
Plot_metrics_v2(fs, LLs, ESSs, SSs, filename='test')

In [ ]:
aa = np.load('budget-ess.npy')
bb = np.load('../aistats/budget/budget-ess.npy')

In [ ]:
import matplotlib.pyplot as plt
bb[:,:,2] = aa[:,:,2]

In [ ]:
np.save('../aistats/budget/budget-ess.npy', bb)